**9.** Simulation of a queuing problem: a clinic has three doctors. Patients come into the
clinic at random, starting at 9 a.m., according to a Poisson process with time parameter
10 minutes: that is, the time after opening at which the first patient appears follows an
exponential distribution with expectation 10 minutes and then, after each patient arrives,
the waiting time until the next patient is independently exponentially distributed, also
with expectation 10 minutes. When a patient arrives, he or she waits until a doctor
is available. The amount of time spent by each doctor with each patient is a random
variable, uniformly distributed between 5 and 20 minutes. The office stops admitting
new patients at 4 p.m. and closes when the last patient is through with the doctor.

**(a)** Simulate this process once. How many patients came to the office? How many had to
wait for a doctor? What was their average wait? When did the office close?


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import random

In [2]:
def poissonprocess(size = 1):
    n = np.random.uniform(0, 1, size = size)
    return  -np.log(1.0 - n) / (1/10)

In [3]:
def queuing(ndoctors = 3, λ = 10, a = 5, b = 20):
    
    # ndoctors: number of doctors
    # λ: rate of patients appearance
    # a, b: minimun and maximun time the doctors spend with each patient
    
    next_patient = 0
    open_time = 0
    time = 0
    patient_arrival_time = 0
    patients = [round(st.expon.rvs(0, λ))]
    doctors = np.repeat([True], ndoctors)
    time_finish_app = np.zeros(ndoctors)
    next_patient_arrival = 0
    patients_waiting = 0
    waiting_time = []
    while next_patient_arrival <= 420:
    
        if np.any(doctors) == False: 
            
            # If doctors are busy, patient_arrival_time is equal to the time the first appoinment finished
            patients_waiting += 1
            patient_arrival_time += patients[next_patient]
            patient_entry_to_app = np.min(time_finish_app) + 1
            waiting_time.append(patient_entry_to_app - patient_arrival_time)
            # The doctor who finishes first his appoinment gets unoccupied
            doctors[np.argmin(time_finish_app)] = True
            # Add arrival time for the next patient
            patients.append(round(st.expon.rvs(0, λ)))
            time_with_doctor = st.randint.rvs(a, b)
            # Time at the end of the appoinment
            time =  patient_entry_to_app + time_with_doctor
            # which doctor is free
            doctor = np.argwhere(doctors == True)[0][0] 
            
            next_patient += 1
            next_patient_arrival =  patient_arrival_time + patients[next_patient]
        
            if next_patient_arrival < time:
                doctors[doctor] = False
                time_finish_app[doctor] = time
            else:
                doctors[doctor] = True
                time_finish_app[doctor] = 0
                
            # If doctors finish appoinments before the arrival of the next patient they become free again   
            time_finish_app = np.where(time_finish_app < next_patient_arrival, 0, time_finish_app)
            doctors = np.where(time_finish_app == 0, True, False)

        else:
            patients.append(round(st.expon.rvs(0, λ)))
            time_with_doctor = st.randint.rvs(a, b)
            patient_arrival_time += patients[next_patient]
            time =  patient_arrival_time + time_with_doctor
            doctor = np.argwhere(doctors == True)[0][0] 
        
            next_patient += 1
            next_patient_arrival =  patient_arrival_time + patients[next_patient]
        
            if next_patient_arrival <= time:
                doctors[doctor] = False
                time_finish_app[doctor] = time
            else:
                doctors[doctor] = True
                time_finish_app[doctor] = 0
            
            time_finish_app = np.where(time_finish_app < next_patient_arrival, 0, time_finish_app)
            doctors = np.where(time_finish_app == 0, True, False)
            
    if len(waiting_time) > 0: 
        Average_waiting_time = sum(waiting_time) / len(waiting_time)
    else:
        Average_waiting_time = 0
        
    return next_patient + 1, patients_waiting, Average_waiting_time, max(420, time)


In [4]:
tot_patients, patients_waiting, waiting_time_avg, closing_time = queuing()
print(tot_patients, 'patients came to the office')
print(patients_waiting, 'patients had to wait for a doctor')
print(f'Their average wait was {round(waiting_time_avg)} minutes')
print(f'The office closed after {closing_time} minutes of being opened')

35 patients came to the office
5 patients had to wait for a doctor
Their average wait was 3 minutes
The office closed after 420 minutes of being opened


**(b)** Simulate the process 100 times and estimate the median and 50% interval for each of
the summaries in (a).

In [5]:
res = {'patients': [], 'pat_waiting': [], 'avg_waiting': [], 'finish_time': []}
for i in range(100):
    tot_patients, patients_waiting, waiting_time_avg, closing_time = queuing()
    res['patients'].append(tot_patients)
    res['pat_waiting'].append(patients_waiting)
    res['avg_waiting'].append(waiting_time_avg)
    res['finish_time'].append(closing_time)

In [6]:
labels = ['Number of patients', '# patients waiting', 'Average waiting time', 'Time opened']
variables = ['patients', 'pat_waiting', 'avg_waiting', 'finish_time']

print("\n{:^20} | {:^10} | {:^10} | {:^16}".format("", "Median", "Mean", "50% interval"))
print("*"*70)
for var, label in zip(variables, labels):
    print("{:<20} | {:^10} | {:^10} | {:^16}".format(label,
                                            np.round(np.median(res[var]), 1), 
                                            np.round(np.mean(res[var]), 2),
                                            str(np.round(st.t.interval(0.5, len(res[var])-1,
                                                                       loc=np.mean(res[var]),
                                                                       scale= st.sem(res[var])), 3))))
    


                     |   Median   |    Mean    |   50% interval  
**********************************************************************
Number of patients   |    42.0    |   42.68    | [42.174 43.186] 
# patients waiting   |    5.0     |    6.47    |  [6.127 6.813]  
Average waiting time |    4.6     |    4.67    |  [4.523 4.823]  
Time opened          |   424.5    |   426.4    | [425.937 426.863]
